In [0]:
%sql
--Create schema
CREATE SCHEMA IF NOT EXISTS `breweries-catalog`.gold_layer;

In [0]:
from pyspark.sql import functions as F

#Read silver table
df_silver = spark.read.table("`breweries-catalog`.silver_layer.breweries_silver")

#Aggregate
df_gold = df_silver.filter(F.col("processing_date") == F.current_date()).groupBy("brewery_type", "state").agg(
    F.count("*").alias("brewery_count")
)

In [0]:
#DQ

if df_gold.count() == 0:
    raise Exception("Data Quality Check Failed: Gold Table is empty")

null_columns = [c for c in df_gold.columns if df_gold.filter(F.col(c).isNull()).count() > 0]
if null_columns:
    raise Exception(f"Data Quality Check Failed: Null values found in columns: {null_columns}")

# DQ Log
null_counts = df_gold.select([
    F.count(F.when(F.col(c).isNull(), c)).alias(c) for c in df_gold.columns
])
df_gold_dq = null_counts.withColumn("dq_layer", F.lit("gold")) \
                   .withColumn("ingestion_timestamp", F.current_timestamp())

# Salvar informações na tabela de dq
df_gold_dq.write.mode("append").format("delta").saveAsTable("`breweries-catalog`.gold_layer.breweries_dq")

In [0]:
# Write table
df_gold.write \
    .format("delta") \
    .mode("overwrite") \
    .saveAsTable("`breweries-pipeline-data-catalog`.gold_layer.breweries_gold")

In [0]:
# df_bronze = spark.read.table("`breweries-pipeline-data-catalog`.bronze_layer.breweries_bronze")

# df_bronze_agg = df_bronze.groupBy("brewery_type", "state").agg(
#     F.count("*").alias("bronze_brewery_count")
# )

# df_gold = spark.read.table("`breweries-pipeline-data-catalog`.gold_layer.breweries_gold")

# df_comparison = df_bronze_agg.join(
#     df_gold,
#     on=["brewery_type", "state"],
#     how="outer"
# ).select(
#     "brewery_type",
#     "state",
#     "bronze_brewery_count",
#     F.col("brewery_count").alias("gold_brewery_count")
# )

# display(df_comparison)